# 일원 분산 분석

주어진 데이터는 4가지 다른 교육 방법을 적용한 대학생들의 학점 결과이다. 이 실험에서는 비슷한 실력을 가진 학생 40명을 무작위로 4개(A, B, C, D)그룹으로 나누었고, 각 그룹은 다른 교육 방법을 적용했다. 학생들의 학점 결과에는 교육 방법에 따른 차이가 있는지 유의수준 0.5하에서 검정하시오.
- 귀무가설(H0): 네 가지 교육 방법에 의한 학생들의 학점 평균은 동일하다.
- 대립가설(H1): 적어도 두 그룹의 학점 평균은 다르다.

## 1. 기초

In [11]:
import pandas as pd
df = pd.DataFrame({
    'A': [3.5, 4.3, 3.8, 3.6, 4.1, 3.2, 3.9, 4.4, 3.5, 3.3],
    'B': [3.9, 4.4, 4.1, 4.2, 4.5, 3.8, 4.2, 3.9, 4.4, 4.3],
    'C': [3.2, 3.7, 3.6, 3.9, 4.3, 4.1, 3.8, 3.5, 4.4, 4.0],
    'D': [3.8, 3.4, 3.1, 3.5, 3.6, 3.9, 3.2, 3.7, 3.3, 3.4]
})
print(df.head(2))

     A    B    C    D
0  3.5  3.9  3.2  3.8
1  4.3  4.4  3.7  3.4


### 일원 분산 분석

In [12]:
# 일원 분산 분석
from scipy import stats

In [13]:
# 정규성, 등분산, 일원 분산 분석

# Shapiro-Wilk(샤피로-윌크) 정규성 검정
from scipy import stats
print(stats.shapiro(df['A']))
print(stats.shapiro(df['B']))
print(stats.shapiro(df['C']))
print(stats.shapiro(df['D']))

# Levene(레빈) 등분산 검정
print(stats.levene(df['A'], df['B'], df['C'], df['D']))

# 일원 분산 분석
print(stats.f_oneway(df['A'], df['B'], df['C'], df['D']))

ShapiroResult(statistic=0.9498823422157768, pvalue=0.667108682193412)
ShapiroResult(statistic=0.9346447100990081, pvalue=0.495096931740429)
ShapiroResult(statistic=0.9871342658078467, pvalue=0.9919546177620264)
ShapiroResult(statistic=0.9752339025839639, pvalue=0.9346854448707619)
LeveneResult(statistic=1.5433829973707245, pvalue=0.22000894224209636)
F_onewayResult(statistic=7.2969837587007, pvalue=0.0006053225519892207)


## 2. 심화

In [14]:
# 데이터 재구조화 (긴 형태)
df_melt = df.melt()

### 분산분석 테이블

In [15]:
# ANOVA 테이블
from statsmodels.formula.api import ols
formula = 'value~variable'
model = ols(formula, data=df_melt).fit()

from statsmodels.stats.anova import anova_lm
anova_lm(model)

,df,sum_sq,mean_sq,F,PR(>F)
variable,3.0,2.35875,0.78625,7.296984,0.000605
Residual,36.0,3.87900,0.10775,NaN,NaN


### 사후검정
- 목적: 어떤 그룹들 간에 통계적으로 유의미한 차이가 있는지 구체적으로 파악하는 것

In [16]:
from scipy import stats
from statsmodels.stats.multicomp import pairwise_tukeyhsd, MultiComparison
# Tukey HSD (투키)
tukey_result = pairwise_tukeyhsd(df_melt['value'], df_melt['variable'], alpha=0.05)
print(tukey_result)

# Bonferroni(본페로니)
mc = MultiComparison(df_melt['value'], df_melt['variable'])
bon_result = mc.allpairtest(stats.ttest_ind, method='bonf')
print(bon_result)

Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower   upper  reject
----------------------------------------------------
     A      B     0.41 0.0397  0.0146  0.8054   True
     A      C     0.09 0.9273 -0.3054  0.4854  False
     A      D    -0.27 0.2722 -0.6654  0.1254  False
     B      C    -0.32 0.1483 -0.7154  0.0754  False
     B      D    -0.68 0.0003 -1.0754 -0.2846   True
     C      D    -0.36 0.0852 -0.7554  0.0354  False
----------------------------------------------------
(<class 'statsmodels.iolib.table.SimpleTable'>, (array([[-2.71993456e+00,  1.40437634e-02],
       [-5.14963436e-01,  6.12844539e-01],
       [ 1.75383847e+00,  9.64687837e-02],
       [ 2.29746611e+00,  3.38010119e-02],
       [ 6.06863400e+00,  9.79510072e-06],
       [ 2.52187810e+00,  2.13149894e-02]]), array([False, False, False, False,  True, False]), array([8.42625806e-02, 1.00000000e+00, 5.78812702e-01, 2.02806071e-01,
       5.87706043e-05, 1.27889936e-01])

### 크루스칼-왈리스 검정 (비모수 검정)

In [18]:
import pandas as pd
from scipy import stats

# 데이터
df = pd.DataFrame({
    'A': [10.5, 11.3, 10.8, 10.6, 11.1, 10.2, 10.9, 11.4, 10.5, 10.3],
    'B': [10.9, 11.4, 11.1, 11.2, 11.5, 10.8, 11.2, 10.9, 11.4, 11.3],
    'C': [10.2, 10.7, 10.6, 10.9, 11.3, 11.1, 10.8, 10.5, 11.4, 11.0],
    'D': [13.8, 10.4, 10.1, 10.5, 10.6, 10.9, 10.2, 10.7, 10.3, 10.4]
})

# 정규성 검정
print(stats.shapiro(df['A']))
print(stats.shapiro(df['B']))
print(stats.shapiro(df['C']))
print(stats.shapiro(df['D']))

# Kruskal-Wallis 검정
print(stats.kruskal( df['A'], df['B'], df['C'], df['D']))

ShapiroResult(statistic=0.9498823422157767, pvalue=0.6671086821934112)
ShapiroResult(statistic=0.9346447100990083, pvalue=0.49509693174043135)
ShapiroResult(statistic=0.9871342658078467, pvalue=0.9919546177620264)
ShapiroResult(statistic=0.5759975003447693, pvalue=2.8656624682936833e-05)
KruskalResult(statistic=11.183607021517561, pvalue=0.010773365310213669)


# 이원 분산 분석

가정에서 재배하고 있는 네 가지 토마토 종자(A, B, C, D)에 대해 세 가지 종류의 비료 (11, 12, 13)를 사용하여 재배된 토마토 수를 조사하였다. 종자 및 비료 종류 간의 토마토 수의 차이가 있는지 유의수준 0.05하에서 검정하시오.
(단, 정규성, 등분산성에 만족한 데이터)
- 종자 (주 효과)
    - 귀무가설(H0): 종자 간의 토마토 수에 차이가 없다.
    - 대립가설(H1): 적어도 하나의 종자에서 토마토 수에 차이가 있다.
- 비료 (주 효과)
    - 귀무가설(H0): 비료 종류 간의 토마토 수에 차이가 없다.
    - 대립가설(H1): 적어도 하나의 비료 종류에서 토마토 수에 차이가 있다.
- 상호작용 효과:
    - 귀무가설(H0): 종자와 비료 간의 상호작용은 토마토 수에 영향을 미치지 않는다.
    - 대립가설(H1): 종자와 비료 간의 상호작용은 토마토 수에 영향을 미친다.


## 1. 기초

In [19]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/main/p3/tomato.csv")
# df = pd.read_csv("tomato.csv")
print(df.head())
print(df.shape)

  종자  비료  토마토수
0  A  11    54
1  A  11    48
2  A  11    56
3  A  11    65
4  A  11    47
(120, 3)


### 이원 분산 분석

In [27]:
# anova 테이블
from statsmodels.stats.anova import anova_lm
from statsmodels.formula.api import ols
formula='토마토수~종자+비료 +종자:비료'
model = ols(formula, data=df).fit()
anova_lm(model)

,df,sum_sq,mean_sq,F,PR(>F)
종자,3.0,4801.000000,1600.333333,18.297521,9.892196e-10
비료,1.0,877.812500,877.812500,10.036530,1.978360e-03
종자:비료,3.0,406.137500,135.379167,1.547867,2.061233e-01
Residual,112.0,9795.716667,87.461756,NaN,NaN


In [32]:
# 범주형 데이터 처리
from statsmodels.stats.anova import anova_lm
from statsmodels.formula.api import ols
formula='토마토수~C(종자)+C(비료) +C(종자):C(비료)'
model = ols(formula, data=df).fit()
anova_lm(model)

,df,sum_sq,mean_sq,F,PR(>F)
C(종자),3.0,4801.000000,1600.333333,18.757977,7.254117e-10
C(비료),2.0,1140.316667,570.158333,6.682993,1.835039e-03
C(종자):C(비료),6.0,725.350000,120.891667,1.417007,2.146636e-01
Residual,108.0,9214.000000,85.314815,NaN,NaN


In [31]:
# 일반표기법 format(지수표기법, '.10f')
format(7.254117e-10, ".11f")

'0.00000000073'

In [ ]:
# formula * 활용
from statsmodels.stats.anova import anova_lm
from statsmodels.formula.api import ols
formula='토마토수~C(종자)*C(비료)'
model = ols(formula, data=df).fit()
anova_lm(model)

## 2. 심화

### 사후검정

In [34]:
# 이원 분산 분석 수행
formula='토마토수~C(종자)*C(비료)'
model = ols(formula, data=df).fit()
anova_lm(model)

,df,sum_sq,mean_sq,F,PR(>F)
C(종자),3.0,4801.000000,1600.333333,18.757977,7.254117e-10
C(비료),2.0,1140.316667,570.158333,6.682993,1.835039e-03
C(종자):C(비료),6.0,725.350000,120.891667,1.417007,2.146636e-01
Residual,108.0,9214.000000,85.314815,NaN,NaN


In [35]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd
# Tukey HSD

tukey_summary1 = pairwise_tukeyhsd(df['토마토수'], df['종자'], alpha=0.05)
tukey_summary2 = pairwise_tukeyhsd(df['토마토수'], df['비료'].astype(str), alpha=0.05)
print(tukey_summary1)
print(tukey_summary2)

Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower   upper  reject
----------------------------------------------------
     A      B   5.7333 0.1106 -0.8444  12.311  False
     A      C     12.1    0.0  5.5223 18.6777   True
     A      D     16.7    0.0 10.1223 23.2777   True
     B      C   6.3667 0.0616  -0.211 12.9444  False
     B      D  10.9667 0.0002   4.389 17.5444   True
     C      D      4.6 0.2679 -1.9777 11.1777  False
----------------------------------------------------
Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower   upper  reject
----------------------------------------------------
    11     12    0.175 0.9973 -5.7831  6.1331  False
    11     13    6.625 0.0254  0.6669 12.5831   True
    12     13     6.45 0.0305  0.4919 12.4081   True
----------------------------------------------------


In [39]:
from scipy import stats
from statsmodels.stats.multicomp import MultiComparison
# Bonferroni
mc1 = MultiComparison(df['토마토수'], df['종자'])
bon_result1 = mc1.allpairtest(stats.ttest_ind, method='bonf', alpha=0.05)
print(bon_result1)

mc2 = MultiComparison(df['토마토수'], df['비료'].astype(str))
bon_result2 = mc2.allpairtest(stats.ttest_ind, method='bonf', alpha=0.05)
print(bon_result2)

(<class 'statsmodels.iolib.table.SimpleTable'>,
 (array([[-2.34572196e+00,  2.24309456e-02],
         [-4.90959632e+00,  7.78628724e-06],
         [-7.01623345e+00,  2.75279700e-09],
         [-2.39444441e+00,  1.98979779e-02],
         [-4.24911814e+00,  7.87969349e-05],
         [-1.76906032e+00,  8.21414155e-02]]),
  array([False,  True,  True, False,  True, False]),
  array([1.34585674e-01, 4.67177235e-05, 1.65167820e-08, 1.19387867e-01,
         4.72781610e-04, 4.92848493e-01]),
  0.008512444610847103,
  0.008333333333333333),
 array([('A', 'B', -2.3457, 0.0224, 0.1346, False),
        ('A', 'C', -4.9096, 0.    , 0.    ,  True),
        ('A', 'D', -7.0162, 0.    , 0.    ,  True),
        ('B', 'C', -2.3944, 0.0199, 0.1194, False),
        ('B', 'D', -4.2491, 0.0001, 0.0005,  True),
        ('C', 'D', -1.7691, 0.0821, 0.4928, False)],
       dtype=[('group1', 'O'), ('group2', 'O'), ('stat', '<f8'), ('pval', '<f8'), ('pval_corr', '<f8'), ('reject', '?')]))

(<class 'statsmodels.iolib.table.SimpleTable'>,
 (array([[-0.06906294,  0.94511627],
         [-2.87223688,  0.00524694],
         [-2.41100168,  0.01826125]]),
  array([False,  True, False]),
  array([1.        , 0.01574082, 0.05478376]),
  0.016952427508441503,
  0.016666666666666666),
 array([('11', '12', -0.0691, 0.9451, 1.    , False),
        ('11', '13', -2.8722, 0.0052, 0.0157,  True),
        ('12', '13', -2.411 , 0.0183, 0.0548, False)],
       dtype=[('group1', 'O'), ('group2', 'O'), ('stat', '<f8'), ('pval', '<f8'), ('pval_corr', '<f8'), ('reject', '?')]))

### [참고] 정규성, 등분산

In [40]:
from scipy.stats import shapiro

cond_tree_A = df['종자'] == 'A'
cond_tree_B = df['종자'] == 'B'
cond_tree_C = df['종자'] == 'C'
cond_tree_D = df['종자'] == 'D'

cond_fert_1 = df['비료'] == 11
cond_fert_2 = df['비료'] == 12
cond_fert_3 = df['비료'] == 13

print(shapiro(df[cond_tree_A & cond_fert_1]['토마토수']))
print(shapiro(df[cond_tree_A & cond_fert_2]['토마토수']))
print(shapiro(df[cond_tree_A & cond_fert_3]['토마토수']))

print(shapiro(df[cond_tree_B & cond_fert_1]['토마토수']))
print(shapiro(df[cond_tree_B & cond_fert_2]['토마토수']))
print(shapiro(df[cond_tree_B & cond_fert_3]['토마토수']))

print(shapiro(df[cond_tree_C & cond_fert_1]['토마토수']))
print(shapiro(df[cond_tree_C & cond_fert_2]['토마토수']))
print(shapiro(df[cond_tree_C & cond_fert_3]['토마토수']))

print(shapiro(df[cond_tree_D & cond_fert_1]['토마토수']))
print(shapiro(df[cond_tree_D & cond_fert_2]['토마토수']))
print(shapiro(df[cond_tree_D & cond_fert_3]['토마토수']))

ShapiroResult(statistic=0.897816553535449, pvalue=0.20729402301632316)
ShapiroResult(statistic=0.9525045795563034, pvalue=0.6981772706403775)
ShapiroResult(statistic=0.9625476904883782, pvalue=0.8144872716038309)
ShapiroResult(statistic=0.9678788472402255, pvalue=0.8705118004375336)
ShapiroResult(statistic=0.9723742113682119, pvalue=0.9119427430913412)
ShapiroResult(statistic=0.9000301149374218, pvalue=0.219263211589448)
ShapiroResult(statistic=0.9434703107703121, pvalue=0.5922336421612504)
ShapiroResult(statistic=0.8953344131173363, pvalue=0.1945690919149648)
ShapiroResult(statistic=0.9231229200955566, pvalue=0.38373908869822115)
ShapiroResult(statistic=0.9585067948704918, pvalue=0.7687207995864245)
ShapiroResult(statistic=0.9392850171805608, pvalue=0.5450853525584016)
ShapiroResult(statistic=0.9591977167835856, pvalue=0.7766866300366541)


In [ ]:
from scipy.stats import levene
print(levene(df[cond_tree_A]['토마토수'],
             df[cond_tree_B]['토마토수'],
             df[cond_tree_C]['토마토수'],
             df[cond_tree_D]['토마토수']))
print(levene(df[cond_fert_1]['토마토수'],
             df[cond_fert_2]['토마토수'],
             df[cond_fert_3]['토마토수']))